In [35]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [36]:
log = pd.read_csv("logs.csv")
contents = pd.read_csv("contents.csv")

In [37]:
log = log[["item_id","timestamp","action"]]

In [38]:
contents = contents[["id","revision_timestamp"]]

In [39]:
log.head()

,item_id,timestamp,action
0,1,2004-12-23T07:10:09Z,delete
1,2,2004-12-23T07:14:09Z,delete
2,3,2004-12-23T07:26:10Z,delete
3,4,2004-12-23T07:55:09Z,delete
4,5,2004-12-23T08:02:22Z,delete


In [40]:
contents.head()

,id,revision_timestamp
0,5,2021-01-19T18:40:34Z
1,7,2021-08-31T18:27:01Z
2,10,2021-08-28T17:17:54Z
3,15,2021-08-29T17:16:00Z
4,17,2020-11-12T19:56:15Z


In [41]:
log["timestamp"] = pd.to_datetime(log["timestamp"])
contents["revision_timestamp"] = pd.to_datetime(contents["revision_timestamp"])

In [42]:
log["timestamp"] = log["timestamp"].apply(lambda x: x.day_name())

In [43]:
contents["revision_timestamp"] = contents["revision_timestamp"].apply(lambda x: x.day_name())

In [44]:
log.head()

,item_id,timestamp,action
0,1,Thursday,delete
1,2,Thursday,delete
2,3,Thursday,delete
3,4,Thursday,delete
4,5,Thursday,delete


In [45]:
contents.head()

,id,revision_timestamp
0,5,Tuesday
1,7,Tuesday
2,10,Saturday
3,15,Sunday
4,17,Thursday


In [46]:
gb1 = log.groupby(["action","timestamp"]).count()
gb1.head()

item_id
action timestamp         
add    Friday         368
       Monday         276
       Saturday       295
       Sunday         325
       Thursday       486

In [47]:
gb1 = gb1.reset_index()
gb1.head()

,action,timestamp,item_id
0,add,Friday,368
1,add,Monday,276
2,add,Saturday,295
3,add,Sunday,325
4,add,Thursday,486


In [57]:
df2 = gb1.reset_index()

In [58]:
df2.head(10)

,index,action,timestamp,item_id
0,0,add,Friday,368
1,1,add,Monday,276
2,2,add,Saturday,295
3,3,add,Sunday,325
4,4,add,Thursday,486
5,5,add,Tuesday,205
6,6,add,Wednesday,231
7,7,addlink,Friday,97
8,8,addlink,Monday,12
9,9,addlink,Saturday,21


In [60]:
df2 = df2.groupby("action").sum()
df2 = df2.reset_index()

In [61]:
df2.head(5)

,action,index,item_id
0,add,21,2186
1,addlink,57,141
2,autocreate,112,1369064
3,block,161,342302
4,byemail,210,6397


In [62]:
df2 = df2.merge(gb1,on="action")

In [63]:
df2.head(5)

,action,index,item_id_x,timestamp,item_id_y
0,add,21,2186,Friday,368
1,add,21,2186,Monday,276
2,add,21,2186,Saturday,295
3,add,21,2186,Sunday,325
4,add,21,2186,Thursday,486


In [64]:
df2["probabilidad"] = df2["item_id_y"]/df2["item_id_x"]

In [66]:
df2 = df2[["action","probabilidad","timestamp"]]

In [68]:
#probabilidad para cada accion por dia de la semana
df2.head(10)

,action,probabilidad,timestamp
0,add,0.17,Friday
1,add,0.13,Monday
2,add,0.13,Saturday
3,add,0.15,Sunday
4,add,0.22,Thursday
5,add,0.09,Tuesday
6,add,0.11,Wednesday
7,addlink,0.69,Friday
8,addlink,0.09,Monday
9,addlink,0.15,Saturday


In [72]:
contents.head()

,revision_timestamp,id
0,Friday,541585
1,Monday,625698
2,Saturday,545963
3,Sunday,548149
4,Thursday,617329


In [71]:
contents = contents.groupby("revision_timestamp").count().reset_index()

In [73]:
contents["id"] = contents["id"]/contents.id.sum()

In [85]:
contents.head(8)

,revision_timestamp,id
0,Friday,0.13
1,Monday,0.15
2,Saturday,0.13
3,Sunday,0.13
4,Thursday,0.15
5,Tuesday,0.15
6,Wednesday,0.15


In [78]:
log.head()

,timestamp,item_id,action
0,Friday,1612898,1612898
1,Monday,1767544,1767544
2,Saturday,1480815,1480814
3,Sunday,1944317,1944316
4,Thursday,1762898,1762898


In [77]:
log = log.groupby("timestamp").count().reset_index()

In [79]:
log["probabilidad"] = log["item_id"]/log["item_id"].sum()

In [82]:
log = log[["timestamp","probabilidad"]]

In [84]:
log.head(8)

,timestamp,probabilidad
0,Friday,0.13
1,Monday,0.15
2,Saturday,0.12
3,Sunday,0.16
4,Thursday,0.14
5,Tuesday,0.15
6,Wednesday,0.15


In [86]:
log = log.rename(columns={'timestamp': 'weekday', 'probabilidad': 'probabilidad'})
contents = contents.rename(columns={'revision_timestamp': 'weekday', 'id': 'probabilidad'})

In [90]:
contents.head()

,weekday,probabilidad
0,Friday,0.13
1,Monday,0.15
2,Saturday,0.13
3,Sunday,0.13
4,Thursday,0.15


In [91]:
log = log.merge(contents,on="weekday")

In [92]:
log.head()

,weekday,probabilidad_x,probabilidad_y
0,Friday,0.13,0.13
1,Monday,0.15,0.15
2,Saturday,0.12,0.13
3,Sunday,0.16,0.13
4,Thursday,0.14,0.15


In [119]:
entropia_plog = (-(log["probabilidad_x"]*np.log2(log["probabilidad_x"]))).sum()
entropia_qcontents = (-(log["probabilidad_y"]*np.log2(log["probabilidad_y"]))).sum()
entropia_plog_qcontents = (-(log["probabilidad_x"]*np.log2(log["probabilidad_y"]))).sum()
entropia_qcontents_plog = (-(log["probabilidad_y"]*np.log2(log["probabilidad_x"]))).sum()

In [120]:
entropia_qcontents_plog


2.8086717583228906

In [121]:
entropia_plog_qcontents

2.8074121856507315

In [122]:
entropia_plog

2.8026413480771266

In [123]:
entropia_qcontents

2.804096796716986

In [125]:
kl1 = entropia_qcontents_plog - entropia_plog

In [126]:
kl1

0.006030410245764006

In [127]:
kl2 = entropia_plog_qcontents - entropia_qcontents

In [128]:
kl2

0.003315388933745389